In [ ]:
#r "nuget: Yueyinqiu.Su.D2lTorchSharp, *-*"

using ScottPlot;
using System.Diagnostics;
using TorchSharp;
using TorchSharp.Modules;
using Yueyinqiu.Su.D2lTorchSharp;

D2l.Notebook.PrepareAll();

3.2.1. 生成数据集

In [ ]:
var true_w = torch.tensor(new float[] { 2, -3.4f });
var true_b = 4.2;
var (features, labels) = D2l.Ch3.synthetic_data(true_w, true_b, 1000);

3.3.2. 读取数据集

In [ ]:
IterableDataLoader load_array(torch.Tensor[] data_arrays, int batch_size, bool is_train = true)
{
    var dataset = torch.utils.data.TensorDataset(data_arrays);
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle: is_train);
}

var batch_size = 10;
var data_iter = load_array([features, labels], batch_size);

In [ ]:
var enumrator = data_iter.GetEnumerator();
enumrator.MoveNext();
(enumrator.Current[0], enumrator.Current[1])

3.3.3. 定义模型

In [ ]:
var net = torch.nn.Sequential(torch.nn.Linear(2, 1));

3.3.4. 初始化模型参数

In [ ]:
((Linear)net[0]).weight.detach().normal_(0, 0.01);
((Linear)net[0]).bias.detach().fill_(0);

3.3.5. 定义损失函数

In [ ]:
var loss = torch.nn.MSELoss();

3.3.6. 定义优化算法

In [ ]:
var trainer = torch.optim.SGD(net.parameters(), learningRate: 0.03);

3.3.7. 训练

In [ ]:
var num_epochs = 3;
foreach (var epoch in D2l.PythonLike.Range(num_epochs))
{
    foreach (var Xy in data_iter)
    {
        using (torch.NewDisposeScope())
        {
            var X = Xy[0];
            var y = Xy[1];
            var l = loss.call(net.call(X), y);
            trainer.zero_grad();
            l.backward();
            trainer.step();
        }
    }
    var lEpoch = loss.call(net.call(features), labels);
    Console.WriteLine($"epoch {epoch + 1}, loss {lEpoch.cstr()}");
}

In [ ]:
var w = ((Linear)net[0]).weight;
Console.WriteLine($"w的估计误差：{(true_w - w.reshape(true_w.shape)).cstr()}");
var b = ((Linear)net[0]).bias;
Console.WriteLine($"b的估计误差：{(true_b - b).cstr()}");